In [1]:
from sklearn import preprocessing
import pandas as pd
import sklearn.svm as svm
from scipy.io import arff
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearnex import patch_sklearn 
import time
from sklearn.preprocessing import MinMaxScaler


In [2]:
patch_sklearn()
clf = svm.SVC(kernel="linear", C=1)
clf2= svm.SVC(kernel="linear", C=1)
pipe = make_pipeline(StandardScaler(),clf)
X = np.array([[-1, -1], [-2, -1],[-3,-1],[3,1], [1, 1], [2, 1],[2.5,1],[1.5,1]])
Z = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
A = np.array([1, 1, 2,2])
Y = np.array([1, 1, 2, 1, 2, 1,2,2])
clf.fit(X,Y)
clf2.fit(Z,A)
R=np.array_split(X,3)
#print(R)
#ARF load, py.io
# Target, and ordinal is good.
# anaconda and jupyter notebook

# svm.Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])

#print(clf.predict([[-.8,-1]]))
vecotr= clf.support_vectors_
vecotr2= clf2.support_vectors_
#print(clf.decision_function(vecotr))
#print(clf2.decision_function(vecotr2))
marginDensity= 0

#initiializes drift detector
class DriftDetector:
    def __init__(self, data, labels,kernel,C,K):
        #needed for other version.
        avAc,stdAc,stdDens= DriftDetector.KXVal(data,labels,kernel,C,K)
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
        svm= DriftDetector.trainSVM(data,labels,kernel,C)
        self.svm= svm
        density = DriftDetector.findMarginDensity(data,svm)
        self.density=density
        self.max=density
        self.min=density
        self.thresh=.075
        self.stdDensThresh=2
    def setDensThresh(self, newTHresh):
        self.stdDensThresh=newTHresh
#needs to be adapted for window stuff.
    def findMarginDensity(batch,svm):
        #vector = svm.support_vectors_[:,0]
        start = time.time()
        marginDensity=0
        for vect in batch:
            if np.abs(svm.decision_function([vect]))<=1:
                marginDensity= marginDensity+1
        end = time.time()
        #print("findmargindensity" + str(end-start),flush=True)
        return marginDensity/batch.size

    def trainSVM(data,labels,kernel,C):
        start = time.time()
        clf= svm.SVC(kernel=kernel,C=C)
        end = time.time()
        #print("makeSVM" + str(end-start),flush=True)
        start = time.time()
        clf.fit(data,labels)
        end = time.time()
        #print("fitsvm" + str(end-start),flush=True)
        return clf
#1st version md3
    def detectDrift(self,batch):
        md2=DriftDetector.findMarginDensity(batch,self.svm)
        if md2>self.max:
            self.max=md2
        if md2<self.min:
            self.min= md2
        if (self.max-self.min)>self.thresh:
            return True
        else: 
            return False
    #multiclass? linear? etc. retrain once found problem.
    def retrain(self,data,labels,kernel,C,K):
        self.svm.fit(data,labels)
        density = self.findMarginDensity(data)
        self.max=density
        self.min=density
        avAc,stdAc,stdDens= DriftDetector.KXVal(data,labels,kernel,C,K)
        self.density=density
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
##cross avlidaton, gets the distribution components.
    def KXVal(data, labels,kernel,C,K):
        #print(data)
        start = time.time()
        splitData=np.array_split(data,K)
        np.asarray(splitData)
        #print(splitData)
        splitLabels=np.array_split(labels,K)
        np.asarray(splitLabels)
        #print(splitLabels)
        accuracies = np.zeros(K)
        densities = np.zeros(K)
        end = time.time()
        #print("splitting Data" +str(end-start),flush=True)
        for i in range(K):
            #print(splitData[:i])
            #print(splitData[i+1])
            #concatenate does no check, can't concat an empty array.
            start = time.time()
            trainData=splitData[:i]+splitData[i+1:]

            #print(K)
            trainData = np.resize(trainData,(data.shape[0]-splitData[i].shape[0],data.shape[1]))
            trainLabels=splitLabels[:i]+splitLabels[i+1:]
            #print(trainLabels)
            trainLabels= np.resize(trainLabels,(labels.shape[0]-splitLabels[i].shape[0],1))
            end = time.time()
            #print("concat"+str(end-start),flush=True)
            dens,acc= DriftDetector.CrossValid(trainData,splitData[i],trainLabels,splitLabels[i],kernel,C)
            accuracies[i]= acc
            densities[i] = dens
        start = time.time()
        averageAccuracy= np.average(accuracies)
        standardDevAccuracy= np.std(accuracies)
        standardDevDensity=np.std(densities)
        #print(averageAccuracy)
        #print("calcstd Density:"+ str(standardDevDensity),flush=True)
        end = time.time()
        #print("calcstd,time"+ str(end-start),flush=True)
        return averageAccuracy, standardDevAccuracy,standardDevDensity

#this is for batch, not sliding window. sliding window forgetting factor sus
#here supposed to query an oracle callback function.
    def calculateMD3drift(self,batch):
        batchMD= DriftDetector.findMarginDensity(batch,self.svm)
        #print("this is batch: " +str(batchMD),flush=True)
        #print("This is selfDensity: "+str(self.density), flush= True)
        if np.abs(batchMD-self.density)>self.stdDensThresh*self.stdDens:
            return True
        else:
            return False

    def checkAccuracy(self,batch,labels):
        if batch.size==labels.size:
            score=self.svm.score(batch,labels)
            if self.avAc-score>self.stdDensThresh*self.stdAc:
                return True
            else:
                return False

            
    # very inefficient, right now, creates new svm. need change the svm, so need to use class svm and fit.
    def CrossValid(trainData,testData,trainLabels,testLabels,kernel,C):
        start = time.time()
        svm= DriftDetector.trainSVM(trainData,trainLabels,kernel,C)
        density = DriftDetector.findMarginDensity(trainData,svm)
        start = time.time()
        score= svm.score(testData,testLabels)
        #print("this is the score on the dataset: " + str(score))
        end = time.time()
        #print("calcscore" + str(end-start),flush=True)
        return density, score




    #need k cross validation. upgraded
    #need the fuzzy algorithm -> basically same algo but using RS and fuzzy. majority voting.
    def FeatureSubspace(data, labels,kernel,C,K):
        
        pass

    def randomSubspace(data):
        
        pass
    ##need virtual margins implementation. inda dumb, just about confidence levels?
    ##ask lorena: if the same but with different way to find classification, not ... if using window.
    #non lineear margins... vrtual.
    
dd= DriftDetector(X,Y,"linear",1,2)
if dd.calculateMD3drift(X):
    print("yes")
else:
    print("no")




Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


no


c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)


In [3]:
#preprocessing.
#LabelEncoder
#iloc and sclice...
#functions to help make arf to usable data
def getCatColumns(pandaFrame):
    names = pandaFrame.select_dtypes(exclude=np.number).columns.tolist()
    return names
    

def fromArftoData(link):
    #load data
    data = arff.loadarff(link)[0]
    df= pd.DataFrame(data)
    #encode labels
    le = preprocessing.LabelEncoder()
    labels= le.fit_transform(df[['class']])
    print("preprocessed",flush=True)
    #encode categorical
    columns = getCatColumns(df)
    print("getcolumns",flush=True)
    ord= preprocessing.OrdinalEncoder()
    scaler= MinMaxScaler()
    df[columns[:]]=ord.fit_transform(df[columns[:]])
    print("get the preprocessed things",flush=True)
    df[df.columns] = scaler.fit_transform(df[df.columns])
    trainingData = df.iloc[:,:-1]
    numpData=trainingData.to_numpy()
    print("tonumpy.",flush=True)
    return numpData,labels

# link = "agraw1_1_abrupt_drift_0_noise_balanced.arff"
# data,labels = fromArftoData("agraw1_1_abrupt_drift_0_noise_balanced.arff")
# drifDet= DriftDetector(data[:30000],labels[:30000],"linear",1,10)
#print(data)
#print(labels)

In [4]:

theSources= np.array(["sea_1_abrupt_drift_0_noise_balanced.arff","agraw1_1_abrupt_drift_0_noise_balanced.arff"
,"agraw2_1_abrupt_drift_0_noise_balanced.arff",
"agraw1_1_gradual_drift_0_noise_balanced_1.arff",
"agraw1_1_gradual_drift_0_noise_balanced_5.arff",
"agraw1_1_gradual_drift_0_noise_balanced_05.arff",
"agraw1_1_gradual_drift_0_noise_balanced_10.arff",
"agraw1_1_gradual_drift_0_noise_balanced_20.arff",
"agraw2_1_gradual_drift_0_noise_balanced_1.arff",
"agraw2_1_gradual_drift_0_noise_balanced_5.arff",
"agraw2_1_gradual_drift_0_noise_balanced_05.arff",
"agraw2_1_gradual_drift_0_noise_balanced_10.arff",
"agraw2_1_gradual_drift_0_noise_balanced_20.arff",
"sea_1_gradual_drift_0_noise_balanced_1.arff",
"sea_1_gradual_drift_0_noise_balanced_5.arff",
"sea_1_gradual_drift_0_noise_balanced_05.arff",
"sea_1_gradual_drift_0_noise_balanced_10.arff",
"sea_1_gradual_drift_0_noise_balanced_20.arff"])
for ind in range(0,theSources.size):
    data,labels = fromArftoData(theSources[ind])
    #print(data)
    drifDet= DriftDetector(data[:30000],labels[:30000],"poly",1,5)
    print("This is the average accuracy of this model:"+str(drifDet.avAc),flush=True)
    batches = np.array_split(data,10)
    batches = np.asarray(batches)
    print("Source:" + theSources[ind])
    for i in range(3,batches.shape[0]):
        if drifDet.calculateMD3drift(batches[i]):
            print("yes")
        else:
            print("no")


# for ind in range(theSources.size):
#     print("Source:" + theSources[ind])
#     for i in range(3,batches.shape[0]):
#         if drifDet.detectDrift(batches[i]):
#             print("yes")
#         else:
#             print("no")

##to change perhaps: expected density etc.



preprocessed
getcolumns
get the preprocessed things
tonumpy.


c:\Python38\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _col

This is the average accuracy of this model:0.8347333333333333
Source:sea_1_abrupt_drift_0_noise_balanced.arff
yes
yes
yes
yes
yes
yes
yes
preprocessed
getcolumns
get the preprocessed things
tonumpy.


c:\Python38\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _col

This is the average accuracy of this model:0.9143000000000001
Source:agraw1_1_abrupt_drift_0_noise_balanced.arff
no
no
no
yes
yes
no
no
preprocessed
getcolumns
get the preprocessed things
tonumpy.


c:\Python38\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _column_or_1d(y, warn=True)
c:\Python38\lib\site-packages\onedal\datatypes\validation.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = _col

This is the average accuracy of this model:0.8807333333333334
Source:agraw2_1_abrupt_drift_0_noise_balanced.arff
yes
no
yes
yes
yes
yes
yes


OSError: [Errno 22] Invalid argument: 'synthetic_data\\gradual_drift\x07graw1_1_gradual_drift_0_noise_balanced_1.arff'